**Studi Kasus 2**

In [ ]:
!pip install python-docx # Install the correct package for working with Word documents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.4 MB/s eta 0:00:00


In [ ]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [ ]:
pip install python-dotenv

In [ ]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 31.2 MB/s eta 0:00:00


In [ ]:
pip install docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53892 sha256=86e8602999c2c4e789451edf4b3f5a0044b3912691c44eba6d363f658c75cb63
  Stored in directory: /root/.cache/pip/wheels/81/f5/1d/e09ba2c1907a43a4146d1189ae4733ca1a3bfe27ee39507767
Successfully built docx


**Ringkasan PADG16**

In [ ]:
import pdfplumber
import openai
import os
import re
from docx import Document

# ------------------------ Configuration ------------------------ #

# Set your OpenAI API key
openai.api_key = ''  # Replace with your actual OpenAI API key

# File path untuk dokumen PADG16
PDF_FILE = 'PADG_162024.pdf'

# Output file path
OUTPUT_FILE = 'PADG16_summary.docx'

# ------------------------ Functions ------------------------ #

def extract_text(pdf_path):
    """
    Mengambil teks dari file PDF menggunakan pdfplumber.

    Args:
        pdf_path (str): Path ke file PDF.

    Returns:
        str: Teks yang diekstrak.
    """
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
            # Debug: Cetak teks dari halaman pertama
            if i == 0:
                print("Teks dari halaman pertama:")
                print(page_text[:500])  # Cetak 500 karakter pertama dari halaman pertama
                print("-" * 50)
    return text

def preprocess_text(text):
    """
    Pra-pemrosesan teks yang diekstrak dengan menghapus karakter dan format yang tidak diinginkan.

    Args:
        text (str): Teks mentah yang diekstrak.

    Returns:
        str: Teks yang telah dipra-pemrosesan.
    """
    # Hapus spasi ganda dan newline
    text = re.sub(r'\s+', ' ', text)
    # Anda dapat menambahkan pra-pemrosesan tambahan di sini jika diperlukan
    return text

def split_into_sections(text):
    """
    Membagi teks menjadi bagian-bagian berdasarkan judul.

    Args:
        text (str): Teks yang telah dipra-pemrosesan.

    Returns:
        dict: Dictionary dengan judul bagian sebagai kunci dan isi bagian sebagai nilai.
    """
    sections = {}
    # Pola regex yang diperbaiki untuk menangkap judul bagian yang lebih kompleks
    pattern = r'(BAB\s+\w+\s+[^BAB]+)'  # Menangkap 'BAB' diikuti oleh kata dan judul

    matches = list(re.finditer(pattern, text, re.IGNORECASE))

    if not matches:
        print("Tidak ada judul bagian yang cocok ditemukan dengan pola regex yang diberikan.")
        return sections

    for i in range(len(matches)):
        start = matches[i].end()
        end = matches[i+1].start() if i+1 < len(matches) else len(text)
        section_title = matches[i].group().strip()
        section_content = text[start:end].strip()
        sections[section_title] = section_content

    # Debug: Cetak judul bagian yang diekstrak
    for title in sections:
        print(f"Judul Bagian: {title}")
        print(f"Konten: {sections[title][:100]}...")  # Cetak 100 karakter pertama dari konten
        print("-" * 50)

    return sections

def generate_section_summary(section, content):
    """
    Menghasilkan ringkasan untuk sebuah bagian menggunakan OpenAI GPT.

    Args:
        section (str): Judul bagian.
        content (str): Konten dari bagian tersebut.

    Returns:
        str: Ringkasan yang dihasilkan.
    """
    prompt = f"""
    Anda adalah ahli dalam dokumen regulasi dari perspektif perbankan. Buat ringkasan dari bagian "{section}" berikut ini:

    {content}

    Ringkasan harus jelas dan ringkas, menyoroti poin-poin utama dan implikasinya. Gunakan bahasa Indonesia.
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",  # Anda dapat menggunakan model yang diinginkan
            messages=[
                {"role": "system", "content": "Anda adalah ChatGPT, asisten AI yang ahli dalam menganalisis dokumen regulasi."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,
            temperature=0.3,
        )
        section_summary = response['choices'][0]['message']['content'].strip()
        return section_summary
    except Exception as e:
        print(f"Error generating summary for section {section}: {e}")
        return "Terjadi kesalahan saat menghasilkan ringkasan."

def generate_summary(sections):
    """
    Menghasilkan ringkasan untuk semua bagian menggunakan OpenAI GPT.

    Args:
        sections (dict): Bagian-bagian dari dokumen.

    Returns:
        dict: Ringkasan untuk setiap bagian.
    """
    summary = {}
    for section, content in sections.items():
        print(f"Menghasilkan ringkasan untuk bagian: {section}")
        section_summary = generate_section_summary(section, content)
        summary[section] = section_summary
    return summary

def save_summary_to_word(summary, output_path):
    """
    Menyimpan ringkasan ke dalam file Word.

    Args:
        summary (dict): Ringkasan untuk setiap bagian.
        output_path (str): Path ke file output Word.
    """
    document = Document()
    document.add_heading('Ringkasan PADG16', 0)

    for section, summary_text in summary.items():
        document.add_heading(section, level=1)
        document.add_paragraph(summary_text)

    document.save(output_path)
    print(f"Ringkasan telah disimpan ke {output_path}")

# ------------------------ Main Process ------------------------ #

def main():
    # Ekstrak teks dari PDF PADG16
    print("Ekstraksi teks dari file PDF PADG16...")
    text = extract_text(PDF_FILE)

    # Pra-pemrosesan teks yang diekstrak
    print("Pra-pemrosesan teks...")
    preprocessed_text = preprocess_text(text)

    # Membagi teks menjadi bagian-bagian
    print("Membagi teks menjadi bagian-bagian...")
    sections = split_into_sections(preprocessed_text)

    if not sections:
        print("Tidak ada bagian yang ditemukan dalam dokumen.")
        return

    # Menghasilkan ringkasan untuk setiap bagian
    print("Menghasilkan ringkasan untuk setiap bagian...")
    summary = generate_summary(sections)

    # Menyimpan ringkasan ke dalam file Word
    print("Menyimpan ringkasan ke dalam file Word...")
    save_summary_to_word(summary, OUTPUT_FILE)

    print(f"Proses selesai. Ringkasan tersedia di {OUTPUT_FILE}")

if __name__ == "__main__":
        main()


Ekstraksi teks dari file PDF PADG16...
Teks dari halaman pertama:
PERATURAN ANGGOTA DEWAN GUBERNUR
NOMOR 16 TAHUN 2024
TENTANG
TRANSAKSI PASAR VALUTA ASING BERDASARKAN PRINSIP SYARIAH
DENGAN RAHMAT TUHAN YANG MAHA ESA
ANGGOTA DEWAN GUBERNUR BANK INDONESIA,
Menimbang : a. bahwa dalam mewujudkan tujuan untuk mencapai
stabilitas nilai rupiah, Bank Indonesia menetapkan dan
melaksanakan kebijakan moneter secara berkelanjutan,
konsisten, dan transparan, yang salah satunya dilakukan
melalui pengaturan, pengembangan, dan pengawasan
pasar valuta asing berdasarkan prins
--------------------------------------------------
Pra-pemrosesan teks...
Membagi teks menjadi bagian-bagian...
Judul Bagian: BAB I KETENTU
Konten: AN UMUM Pasal 1 Dalam Peraturan Anggota Dewan Gubernur ini yang dimaksud dengan: 1. Bank Umum Syaria...
--------------------------------------------------
Judul Bagian: BAB II KER
Konten: ANGKA PENGATURAN, PENGEMBANGAN, DAN PENGAWASAN PASAR VALUTA ASING BERDASARKAN PRINSIP SYARIAH Pas

**PADG16 Action Plan**

In [ ]:
import pdfplumber
import openai
import os
import re
from docx import Document

# ------------------------ Configuration ------------------------ #

# Gantilah API key dengan method yang lebih aman (ENV variable)
openai.api_key = ''  # Replace with your actual OpenAI API key
if not openai.api_key:
    raise ValueError("API key OpenAI tidak ditemukan. Harap set variabel lingkungan 'OPENAI_API_KEY'.")

# File path untuk dokumen PADG16
PDF_FILE = 'PADG_162024.pdf'

# Output file path
OUTPUT_FILE_ACTION_PLAN = 'PADG16_action_plan.docx'

# ------------------------ Functions ------------------------ #

def extract_text(pdf_path):
    """
    Mengambil teks dari file PDF menggunakan pdfplumber.
    """
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

def preprocess_text(text):
    """
    Menghapus karakter/format yang tidak diinginkan agar teks lebih rapi.
    """
    # Menghapus spasi ganda dan newline
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

def split_into_sections(text):
    """
    Membagi teks menjadi bagian-bagian (misalnya BAB) agar lebih terstruktur.
    """
    sections = {}
    # Pola regex untuk menangkap "BAB" + spasi + angka Romawi/Arab
    pattern = r'(BAB\s+[IVXLC0-9]+\s.*?)(?=BAB\s+[IVXLC0-9]+|$)'
    matches = re.findall(pattern, text, flags=re.IGNORECASE | re.DOTALL)

    if not matches:
        # Jika tidak ada match, simpan seluruh teks sebagai satu bagian
        sections["Dokumen PADG16 Lengkap"] = text
        return sections

    for section_text in matches:
        lines = section_text.split('\n', 1)
        first_line = lines[0].strip()
        title = first_line if first_line else "Bagian Tanpa Judul"
        sections[title] = section_text

    return sections

def split_text_into_chunks(text, max_tokens=2000):
    """
    Membagi teks panjang menjadi beberapa bagian kecil (chunk)
    agar tetap dalam batas token maksimum OpenAI.
    """
    chunks = []
    current_chunk = ""
    words = text.split()

    for word in words:
        if len((current_chunk + " " + word).split()) > max_tokens:
            chunks.append(current_chunk.strip())
            current_chunk = word
        else:
            current_chunk += " " + word

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

def generate_action_plan_gpt(content, section_title=""):
    """
    Fungsi untuk meminta GPT membuat rencana aksi (action plan)
    berdasarkan konten regulasi.
    """
    # Membagi teks menjadi chunk jika terlalu panjang
    chunks = split_text_into_chunks(content)

    full_action_plan = ""
    for i, chunk in enumerate(chunks):
        print(f"Mengirim chunk {i + 1} dari {len(chunks)} untuk {section_title} ke GPT...")
        prompt = f"""
        Anda adalah analis kepatuhan (compliance) di sebuah bank.
        Berikut ini adalah isi dari bagian "{section_title}" pada regulasi:

        {chunk}

        Berdasarkan regulasi di atas, buat action plan (rencana aksi) yang mencakup:
        1. Tindakan segera yang harus diambil.
        2. Penyesuaian prosedur internal atau pedoman.
        3. Koordinasi dengan unit bisnis terkait.
        4. Rekomendasi timeline implementasi.

        Berikan penjelasan dalam Bahasa Indonesia dengan format poin-poin yang mudah dipahami.
        """
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-16k",
                messages=[
                    {
                        "role": "system",
                        "content": (
                            "Anda adalah ChatGPT, asisten AI yang ahli dalam compliance "
                            "dan mampu menyusun rencana aksi berdasarkan regulasi."
                        ),
                    },
                    {"role": "user", "content": prompt},
                ],
                max_tokens=700,
                temperature=0.3,
            )
            action_plan = response['choices'][0]['message']['content'].strip()
            full_action_plan += f"\n\nChunk {i + 1}:\n{action_plan}"
        except Exception as e:
            print(f"Error generating action plan for chunk {i + 1}: {e}")
            full_action_plan += f"\n\nChunk {i + 1}:\nError generating action plan."

    return full_action_plan.strip()

def save_to_word(data_dict, output_path, doc_title="Action Plan PADG16"):
    """
    Menyimpan action plan per bagian ke dalam file Word.
    data_dict diharapkan berbentuk {judul: teks}, misalnya:
    {
        "BAB V": "Daftar rencana aksi untuk BAB V...",
        "BAB VI": "Daftar rencana aksi untuk BAB VI...",
    }
    """
    document = Document()
    document.add_heading(doc_title, 0)

    for title, text_content in data_dict.items():
        document.add_heading(title, level=1)
        document.add_paragraph(text_content)

    document.save(output_path)
    print(f"Action plan telah disimpan ke: {output_path}")

# ------------------------ Main Process ------------------------ #

def main():
    # 1. Ekstrak teks dari PDF
    print("Ekstraksi teks dari file PDF PADG16...")
    raw_text = extract_text(PDF_FILE)

    # 2. Pra-pemrosesan teks
    print("Pra-pemrosesan teks...")
    preprocessed_text = preprocess_text(raw_text)

    # 3. Membagi teks menjadi beberapa bagian (BAB)
    print("Membagi teks menjadi bagian-bagian...")
    sections = split_into_sections(preprocessed_text)

    # 4. Menghasilkan action plan untuk setiap bagian
    action_plans = {}
    for title, content in sections.items():
        print(f"Membuat action plan untuk: {title[:60]}...")
        plan_text = generate_action_plan_gpt(content, section_title=title)
        action_plans[title] = plan_text

    # 5. Menyimpan action plan ke dalam file Word
    print("Menyimpan Action Plan ke Word...")
    save_to_word(action_plans, OUTPUT_FILE_ACTION_PLAN)

    print("Proses selesai.")

if __name__ == "__main__":
    main()

Ekstraksi teks dari file PDF PADG16...
Pra-pemrosesan teks...
Membagi teks menjadi bagian-bagian...
Membuat action plan untuk: BAB I KETENTUAN UMUM Pasal 1 Dalam Peraturan Anggota Dewan G...
Mengirim chunk 1 dari 1 untuk BAB I KETENTUAN UMUM Pasal 1 Dalam Peraturan Anggota Dewan Gubernur ini yang dimaksud dengan: 1. Bank Umum Syariah yang selanjutnya disingkat BUS adalah bank umum syariah sebagaimana dimaksud dalam Undang-Undang mengenai perbankan syariah, namun tidak termasuk kantor bank umum syariah berbadan hukum Indonesia yang beroperasi di luar negeri. 2. Unit Usaha Syariah yang selanjutnya disingkat UUS adalah unit usaha syariah sebagaimana dimaksud dalam Undang-Undang mengenai perbankan syariah. 3. Bank adalah BUS dan UUS. 4. Prinsip Syariah adalah prinsip hukum Islam berdasarkan fatwa dan/atau pernyataan kesesuaian syariah yang dikeluarkan oleh lembaga yang memiliki kewenangan dalam penetapan fatwa di bidang syariah. 5. Pasar Valuta Asing Berdasarkan Prinsip Syariah adalah bagi

**Gap Analysis PADG16 dengan PADG 24/13/22**

In [ ]:
import pdfplumber
import openai
import os
import re
import openpyxl
from openpyxl.utils import get_column_letter


# Mengambil OpenAI API key dari environment variable
openai.api_key = ''  # Replace with your actual OpenAI API key

# Pastikan Anda telah menempatkan PDF Anda di lokasi yang benar,
# atau tuliskan path lengkap di sini.
PDF_FILE_1 = 'PADG_241322.pdf'
PDF_FILE_2 = 'PADG_162024.pdf'

# Nama file output XLSX
OUTPUT_XLSX = 'gap_analysis_summary.xlsx'

# ------------------------ Fungsi-fungsi ------------------------ #

def extract_text(pdf_path):
    """
    Ekstraksi teks dari file PDF menggunakan pdfplumber.
    """
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

def preprocess_text(text):
    """
    Membersihkan teks hasil ekstraksi dengan menghilangkan spasi/newline berlebih.
    """
    # Menghapus spasi berlebih
    text = re.sub(r'\s+', ' ', text)
    return text

def split_into_sections(text):
    """
    Memisahkan teks ke dalam 'bagian' (section) berdasarkan pola tertentu.
    Pola default: BAB \d+, BAB \w+, KEPALA BAGIAN, Mekanisme Transaksi, dsb.
    Silakan modifikasi jika struktur dokumen berbeda.
    """
    sections = {}
    # Pola regex contoh (sesuaikan dengan dokumen Anda)
    pattern = r'(?:BAB\s+\d+|BAB \w+|KEPALA BAGIAN\s+\w+|Mekanisme Transaksi|Bagian\s+\w+)'
    matches = list(re.finditer(pattern, text, re.IGNORECASE))

    for i in range(len(matches)):
        start = matches[i].start()
        end = matches[i+1].start() if i+1 < len(matches) else len(text)
        section_title = matches[i].group().strip()
        section_content = text[start:end].strip()
        sections[section_title] = section_content
    return sections

def compare_sections(sections1, sections2):
    """
    Membandingkan bagian-bagian dari dua dokumen dan mengidentifikasi perbedaan.
    Menghasilkan dict dengan key = nama bagian, value = isi dokumen 1 & 2.
    """
    differences = {}
    for section in sections1:
        if section in sections2:
            # Cek apakah isi teks sama persis; jika berbeda, masukkan ke differences
            if sections1[section] != sections2[section]:
                differences[section] = {
                    'Document 1': sections1[section],
                    'Document 2': sections2[section]
                }
        else:
            differences[section] = {
                'Document 1': sections1[section],
                'Document 2': 'Bagian tidak ditemukan di Dokumen 2'
            }
    # Cek bagian yang hanya ada di Dokumen 2
    for section in sections2:
        if section not in sections1:
            differences[section] = {
                'Document 1': 'Bagian tidak ditemukan di Dokumen 1',
                'Document 2': sections2[section]
            }
    return differences

def generate_summary(section_name, doc1_text, doc2_text):
    """
    Membuat ringkasan perbedaan dengan memanggil OpenAI ChatCompletion.
    Output dalam bahasa Indonesia.
    """
    prompt = f"""
Anda adalah ahli dalam dokumen regulasi dari perspektif perbankan.
Jelaskan perbedaan antara dua versi bagian "{section_name}" berikut:

Dokumen 1 (PADG 241322):
{doc1_text}

Dokumen 2 (PADG 162024):
{doc2_text}

Berikan ringkasan yang jelas dan ringkas, soroti perubahan kunci dan implikasinya.
Gunakan bahasa Indonesia.
    """.strip()

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=[
                {
                    "role": "system",
                    "content": "Anda adalah ChatGPT, asisten AI yang ahli dalam menganalisis dokumen regulasi."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            max_tokens=500,
            temperature=0.3,
        )
        summary = response['choices'][0]['message']['content'].strip()
        return summary
    except Exception as e:
        print(f"Terjadi error saat menghasilkan ringkasan untuk bagian {section_name}: {e}")
        return "Error: Tidak dapat menghasilkan ringkasan."

# ------------------------ Main Process ------------------------ #

def main():
    print("1. Mengambil teks dari file PDF...")
    text1 = extract_text(PDF_FILE_1)
    text2 = extract_text(PDF_FILE_2)

    print("2. Memproses teks (preprocessing)...")
    preprocessed_text1 = preprocess_text(text1)
    preprocessed_text2 = preprocess_text(text2)

    print("3. Memisahkan teks ke dalam bagian...")
    sections1 = split_into_sections(preprocessed_text1)
    sections2 = split_into_sections(preprocessed_text2)

    print("4. Membandingkan bagian untuk mengidentifikasi perbedaan...")
    differences = compare_sections(sections1, sections2)

    if not differences:
        print("Tidak ada perbedaan yang ditemukan antara kedua dokumen.")
        return

    print("5. Membuat workbook Excel...")
    # Membuat workbook baru
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Gap Analysis"

    # Menulis header
    ws.cell(row=1, column=1, value="Bagian")
    ws.cell(row=1, column=2, value="Isi Dokumen 1 (PADG 241322)")
    ws.cell(row=1, column=3, value="Isi Dokumen 2 (PADG 162024)")
    ws.cell(row=1, column=4, value="Ringkasan Perbedaan")

    # Mengatur lebar kolom (opsional, agar rapi)
    ws.column_dimensions[get_column_letter(1)].width = 30
    ws.column_dimensions[get_column_letter(2)].width = 50
    ws.column_dimensions[get_column_letter(3)].width = 50
    ws.column_dimensions[get_column_letter(4)].width = 50

    print("6. Menghasilkan ringkasan dan memasukkan data ke dalam Excel...")
    row_idx = 2
    for section, content_dict in differences.items():
        doc1_text = content_dict['Document 1']
        doc2_text = content_dict['Document 2']

        # Panggil OpenAI API untuk membuat ringkasan perbedaan
        summary = generate_summary(section, doc1_text, doc2_text)

        # Tulis data ke baris Excel
        ws.cell(row=row_idx, column=1, value=section)
        ws.cell(row=row_idx, column=2, value=doc1_text)
        ws.cell(row=row_idx, column=3, value=doc2_text)
        ws.cell(row=row_idx, column=4, value=summary)

        row_idx += 1

    print(f"7. Menyimpan workbook Excel sebagai {OUTPUT_XLSX}...")
    wb.save(OUTPUT_XLSX)
    print("Proses selesai. File Excel berhasil dibuat.")

if __name__ == "__main__":
    main()


1. Mengambil teks dari file PDF...
2. Memproses teks (preprocessing)...
3. Memisahkan teks ke dalam bagian...
4. Membandingkan bagian untuk mengidentifikasi perbedaan...
5. Membuat workbook Excel...
6. Menghasilkan ringkasan dan memasukkan data ke dalam Excel...


**PADG16 Combined Report (Ringkasan, Action Plan, Gap Analysis)**

In [ ]:
import pdfplumber
import openai
import os
import re
from docx import Document
from docx.shared import Inches

# ------------------------ Configuration ------------------------ #

# Menggunakan variabel lingkungan untuk keamanan
openai.api_key = ''  # Replace with your actual OpenAI API key
if not openai.api_key:
    raise ValueError("API key OpenAI tidak ditemukan. Harap set variabel lingkungan 'OPENAI_API_KEY'.")

# File path untuk dokumen PADG16 dan PADG24
PDF_FILE_16 = "PADG_162024.pdf"
PDF_FILE_24 = "PADG_241322.pdf"  # Gantilah dengan nama file sebenarnya jika berbeda

# Output file path (Dokumen Word) - semua laporan jadi satu file
OUTPUT_FILE = "Combined_Report_PADG16_24.docx"

# ------------------------ Functions ------------------------ #

def extract_text(pdf_path):
    """
    Mengambil teks dari file PDF menggunakan pdfplumber.
    """
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

def preprocess_text(text):
    """
    Menghapus karakter/format yang tidak diinginkan agar teks lebih rapi.
    """
    # Menghapus spasi ganda dan newline
    text = re.sub(r"\s+", " ", text)
    text = text.strip()
    return text

def split_into_sections(text):
    """
    Membagi teks menjadi bagian-bagian berdasarkan pola (misalnya BAB).
    Sesuaikan pola regex dengan format PDF sebenarnya (BAB I / BAB 1 / dsb).
    """
    sections = {}
    # Contoh pola: menangkap "BAB" + spasi + huruf/angka (Romawi/Arab) + isinya
    pattern = r"(BAB\s+[IVXLC0-9]+\s.*?)(?=BAB\s+[IVXLC0-9]+|$)"
    matches = re.findall(pattern, text, flags=re.IGNORECASE | re.DOTALL)

    if not matches:
        # Jika tidak ada match, simpan keseluruhan teks sebagai satu bagian
        sections["Dokumen Lengkap"] = text
        return sections

    for section_text in matches:
        lines = section_text.split('\n', 1)
        first_line = lines[0].strip()
        title = first_line if first_line else "Bagian Tanpa Judul"
        sections[title] = section_text

    return sections

def generate_summary_gpt(section_title, content):
    """
    Membuat ringkasan singkat untuk suatu bagian menggunakan GPT-3.5-turbo.
    """
    prompt = f"""
    Anda adalah ahli dokumen regulasi perbankan. Ringkaslah bagian berikut:

    Judul: {section_title}
    Isi:
    {content}

    Ringkasan harus ringkas, jelas, dan dalam Bahasa Indonesia.
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "Anda adalah ChatGPT, asisten AI yang berpengalaman "
                        "dalam menganalisis dokumen regulasi perbankan."
                    ),
                },
                {"role": "user", "content": prompt},
            ],
            max_tokens=500,
            temperature=0.3,
        )
        return response["choices"][0]["message"]["content"].strip()
    except Exception as e:
        print(f"Error generating summary: {e}")
        return "Terjadi kesalahan saat menghasilkan ringkasan."

def generate_action_plan_gpt(content):
    """
    Membuat rencana aksi (action plan) berdasarkan regulasi menggunakan GPT-3.5-turbo.
    """
    prompt = f"""
    Anda adalah analis kepatuhan (compliance) di sebuah bank.
    Berikut ini adalah isi atau ringkasan dari regulasi yang baru:

    {content}

    Buat action plan (rencana aksi) yang mencakup:
    1. Tindakan segera yang harus diambil
    2. Penyesuaian prosedur internal atau pedoman
    3. Koordinasi dengan unit bisnis terkait
    4. Rekomendasi timeline implementasi

    Berikan penjelasan dalam Bahasa Indonesia, format poin-poin agar mudah dipahami.
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "Anda adalah ChatGPT, asisten AI yang ahli dalam compliance "
                        "dan mampu menyusun rencana aksi berdasarkan regulasi."
                    ),
                },
                {"role": "user", "content": prompt},
            ],
            max_tokens=700,
            temperature=0.3,
        )
        return response["choices"][0]["message"]["content"].strip()
    except Exception as e:
        print(f"Error generating action plan: {e}")
        return "Terjadi kesalahan saat menghasilkan action plan."

def compare_sections(sections16, sections24):
    """
    Membandingkan isi sections dari PADG16 dengan PADG24.
    Mengembalikan dictionary: { section_title: {"PADG16": "...", "PADG24": "..."} }
    """
    differences = {}

    # Loop semua section di PADG16
    for section_title_16, content_16 in sections16.items():
        # Periksa apakah judul yang sama (atau mirip) ada di PADG24.
        # Karena judul mungkin tidak persis sama, Anda bisa modifikasi logika pencocokan
        # misalnya pakai 'closest match' atau fuzzy matching.
        # Untuk contoh sederhana ini, kita cari judul persis.
        if section_title_16 in sections24:
            content_24 = sections24[section_title_16]
            differences[section_title_16] = {
                "PADG16": content_16,
                "PADG24": content_24,
            }
        else:
            # Bagian ini tidak ada di PADG24
            differences[section_title_16] = {
                "PADG16": content_16,
                "PADG24": "Tidak ditemukan di PADG24",
            }

    # Loop untuk mencari section yang hanya ada di PADG24
    for section_title_24, content_24 in sections24.items():
        if section_title_24 not in differences:
            differences[section_title_24] = {
                "PADG16": "Tidak ditemukan di PADG16",
                "PADG24": content_24,
            }

    return differences

def save_to_word(summaries16, summaries24, action_plan16, comparison_result, output_path):
    """
    Menyimpan ringkasan, action plan, dan perbandingan ke dalam file Word dengan tabel.
    """
    document = Document()
    document.add_heading("Laporan PADG16 & PADG24", 0)

    # 1. Ringkasan PADG16
    document.add_heading("Ringkasan PADG16", level=1)
    for title, summary in summaries16.items():
        document.add_heading(title, level=2)
        document.add_paragraph(summary)

    # 2. Ringkasan PADG24
    document.add_heading("Ringkasan PADG24", level=1)
    for title, summary in summaries24.items():
        document.add_heading(title, level=2)
        document.add_paragraph(summary)

    # 3. Action Plan untuk PADG16
    document.add_heading("Action Plan Terkait PADG16", level=1)
    document.add_paragraph(action_plan16)

    # 4. Perbandingan PADG16 vs PADG24 dalam Tabel
    document.add_heading("Perbandingan PADG16 vs PADG24", level=1)
    table = document.add_table(rows=1, cols=3)
    table.style = "Light Grid"

    # Header tabel
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = "Bagian/BAB"
    hdr_cells[1].text = "PADG16"
    hdr_cells[2].text = "PADG24"

    for section_title, contents in comparison_result.items():
        row_cells = table.add_row().cells
        row_cells[0].text = section_title
        # Batasi panjang teks untuk mencegah tabel menjadi terlalu lebar
        padg16_content = contents["PADG16"][:500] + "..." if len(contents["PADG16"]) > 500 else contents["PADG16"]
        padg24_content = contents["PADG24"][:500] + "..." if len(contents["PADG24"]) > 500 else contents["PADG24"]
        row_cells[1].text = padg16_content
        row_cells[2].text = padg24_content

    # Simpan dokumen
    document.save(output_path)
    print(f"Hasil telah disimpan ke: {output_path}")

# ------------------------ Main Process ------------------------ #

def main():
    # 1. Ekstraksi teks dari PDF PADG16 dan PADG24
    print("Ekstraksi teks dari PDF PADG16...")
    raw_text_16 = extract_text(PDF_FILE_16)
    text_16 = preprocess_text(raw_text_16)

    print("Ekstraksi teks dari PDF PADG24...")
    raw_text_24 = extract_text(PDF_FILE_24)
    text_24 = preprocess_text(raw_text_24)

    # 2. Membagi teks menjadi sections (BAB)
    print("Membagi teks PADG16 menjadi bagian-bagian (BAB)...")
    sections16 = split_into_sections(text_16)

    print("Membagi teks PADG24 menjadi bagian-bagian (BAB)...")
    sections24 = split_into_sections(text_24)

    # 3. Membuat ringkasan untuk PADG16
    print("Membuat ringkasan PADG16...")
    summaries16 = {}
    for title, content in sections16.items():
        print(f"Membuat ringkasan untuk: {title[:50]}...")
        summaries16[title] = generate_summary_gpt(title, content)

    # 4. Membuat ringkasan untuk PADG24
    print("Membuat ringkasan PADG24...")
    summaries24 = {}
    for title, content in sections24.items():
        print(f"Membuat ringkasan untuk: {title[:50]}...")
        summaries24[title] = generate_summary_gpt(title, content)

    # 5. Membuat Action Plan untuk PADG16
    # Gabungkan semua ringkasan PADG16 menjadi satu string
    combined_summary_16 = "\n\n".join(summaries16.values())
    print("Membuat Action Plan untuk PADG16...")
    action_plan_16 = generate_action_plan_gpt(combined_summary_16)

    # 6. Membandingkan PADG16 dan PADG24
    print("Membandingkan PADG16 dengan PADG24...")
    comparison_result = compare_sections(sections16, sections24)

    # 7. Menyimpan semua hasil ke satu file Word dengan tabel
    print(f"Menyimpan hasil ke {OUTPUT_FILE}...")
    save_to_word(summaries16, summaries24, action_plan_16, comparison_result, OUTPUT_FILE)

    print("Proses selesai.")

if __name__ == "__main__":
    main()


Ekstraksi teks dari PDF PADG16...
Ekstraksi teks dari PDF PADG24...
Membagi teks PADG16 menjadi bagian-bagian (BAB)...
Membagi teks PADG24 menjadi bagian-bagian (BAB)...
Membuat ringkasan PADG16...
Membuat ringkasan untuk: BAB I KETENTUAN UMUM Pasal 1 Dalam Peraturan Anggo...
Membuat ringkasan untuk: BAB II KERANGKA PENGATURAN, PENGEMBANGAN, DAN PENG...
Membuat ringkasan untuk: BAB III PRODUK PASAR VALUTA ASING BERDASARKAN PRIN...
Membuat ringkasan untuk: BAB IV KURS ACUAN Pasal 7 (1) Dalam hal Transaksi ...
Membuat ringkasan untuk: BAB V TRANSAKSI PASAR VALUTA ASING BERDASARKAN PRI...
Membuat ringkasan untuk: BAB VI UNDERLYING TRANSAKSI Bagian Kesatu Umum Pas...
Membuat ringkasan untuk: BAB VII PENYELESAIAN TRANSAKSI Bagian Kesatu Umum ...
Membuat ringkasan untuk: BAB VIII LARANGAN DAN BATASAN TRANSAKSI PASAR VALU...
Membuat ringkasan untuk: BAB IX DATA DAN INFORMASI Bagian Kesatu Pelaporan ...
Membuat ringkasan untuk: BAB X PRINSIP KEHATI-HATIAN DAN MANAJEMEN RISIKO P...
Membuat ring